In [5]:
import pickle
import pandas as pd
import numpy as np

train_df = pd.read_pickle("data/train_df_new_interviews.pkl")
# train_df['annotator'] = ''

Draw subset from data and split into parts.

In [35]:
subset = train_df.sample(n=210, random_state=2025)
parts = np.array_split(subset.sample(frac=1, random_state=2025), 3)
part1, part2, part3 = parts

/home/sc.uni-leipzig.de/pk35momo/.local/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [36]:
# Combine the first 3 parts to get all used IDs
used_ids = pd.concat([part1, part2, part3]).index

# Select remaining rows that are not in the used_ids
remaining_df = train_df[~train_df.index.isin(used_ids)]

# Draw 70 new examples from the remaining rows
part4 = remaining_df.sample(n=70, random_state=2025)

In [18]:
records = dataset1.records(status_filter).to_list(flatten=True)
ids = {r['small-text-id'] for r in records}
set(part1["id"]) & ids

In [28]:
len(part1.index.intersection(ids))

70

In [3]:
train_df.iloc[2614]

id                                                                       462
model                                             meta-llama-3.1-8b-instruct
interviewer_temperature                                                  1.0
child_temperature                                                        0.0
child_sex                                                              Junge
child_name                                                              Timo
child_health_status                                                 schlecht
interviewer_prompt         Du bist eine freundliche und geduldige Person,...
child_prompt               Du bist ein Kind, das zu seinem Gesundheitszus...
num_questions                                                             20
stop_on_phrase                                                          True
interview_id                                  interview_20251011_062601.json
speech_index                                                               8

In [6]:
codes_health = ["physical_health", "mental_health", "daily_functioning", "health_unspecific", "health_none"]
codes_freq = ["freq_mentioned", "freq_not_mentioned"]
codes_neg = ["health_none", "freq_not_mentioned"]

codes_combined = codes_health + codes_freq
code_full = {label: idx for idx, label in enumerate(codes_combined)}
code_pos = {k: v for k, v in code_full.items() if k not in codes_neg}
code_neg = {k: v for k, v in code_full.items() if k in codes_neg}
code_pos_new = {i: k for k, i in enumerate(code_pos.keys())}

full_to_pos = {
    code_full[label]: new_idx
    for new_idx, label in enumerate(code_pos.keys())
}

num_classes = len(code_pos)
code_pos_new

{'physical_health': 0,
 'mental_health': 1,
 'daily_functioning': 2,
 'health_unspecific': 3,
 'freq_mentioned': 4}

Logging to Argilla

In [7]:
import argilla as rg

client = rg.Argilla(
    api_url="https://ch01.informatik.uni-leipzig.de/argilla", 
    api_key="tjP8Z66OAr3IWVq6nKJ5E7I5y" 
)

In [8]:
dataset1 = client.datasets(name="train_subset_1")
dataset2 = client.datasets(name="train_subset_2")
dataset3 = client.datasets(name="train_subset_3")
datasets = [dataset1, dataset2, dataset3]

In [38]:
settings = rg.Settings(
    guidelines="Active Learning Experiments",
    fields=[
      # Text to be labeld
        rg.TextField(
            name = "childPart",
            title = "Speech Part of the Child",
            use_markdown=False,
        ),
        rg.CustomField(
            name="interview",
            title="Interview Context",
            template="""
                       <a href="javascript:void(0)" 
                       onclick="window.open('', '_blank').document.write(record.fields.interview.text);">
                       Open Interview Context
                    </a>
                """,
            advanced_mode=True,
            )
    ],
    questions=[
      rg.MultiLabelQuestion(
          name="health",
          title="What health Dimension does the childs answer refer to?",
          labels=dict(zip(codes_health, ["Physical health", "Mental health", "Daily functioning", "Unspecific", "None"])),
      ),
      rg.LabelQuestion(
          name="frequency",
          title="The concept 'frequency' is mentioned.",
          labels=dict(zip(codes_freq, ["Yes", "No"])),
      ),
    ],
    metadata=[
        # later we only want to train on the latest batches
        rg.IntegerMetadataProperty(
            name="batch_id",
            min=0,
            max=1000,
            title="batch_id"
        ),
        rg.IntegerMetadataProperty(
            name="small-text-id",
            min=0,
            max=100000,
            title="Small Text ID"
        ),
    ],
)


dataset = rg.Dataset(
    name="train_subset_4",
    settings=settings,
)
dataset.create()

/home/sc.uni-leipzig.de/pk35momo/.local/lib/python3.12/site-packages/argilla/datasets/_resource.py:264: UserWarning: Workspace not provided. Using default workspace: default id: c0cca5f2-fd97-4375-8f7c-61e05c36dccc
  warnings.warn(f"Workspace not provided. Using default workspace: {workspace.name} id: {workspace.id}")


Dataset(id=UUID('a518afe5-cb19-470f-915a-c2a2fbf34115') inserted_at=datetime.datetime(2025, 12, 18, 11, 12, 31, 891310) updated_at=datetime.datetime(2025, 12, 18, 11, 12, 32, 52778) name='train_subset_4' status='ready' guidelines='Active Learning Experiments' allow_extra_metadata=False distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('c0cca5f2-fd97-4375-8f7c-61e05c36dccc') last_activity_at=datetime.datetime(2025, 12, 18, 11, 12, 32, 52778))

In [42]:
current_batch_id = 0

records = [
    rg.Record(
        fields={"childPart":row.get('childPart', ""),
                "interview": {"text": row.get('interview', "")}
               },
        metadata={"batch_id":current_batch_id,
                  "small-text-id": i}
    )  
    for i, row in part4.iterrows() 
]

dataset.records.log(records)

In [5]:
settings = rg.Settings(
    guidelines="Active Learning Experiments",
    fields=[
      # Text to be labeld
        rg.TextField(
            name = "childPart",
            title = "Speech Part of the Child",
            use_markdown=False,
        ),
        rg.CustomField(
            name="interview",
            title="Interview Context",
            template="""
                       <a href="javascript:void(0)" 
                       onclick="window.open('', '_blank').document.write(record.fields.interview.text);">
                       Open Interview Context
                    </a>
                """,
            advanced_mode=True,
            )
    ],
    questions=[
      rg.MultiLabelQuestion(
          name="health",
          title="What health Dimension does the childs answer refer to?",
          labels=dict(zip(codes_health, ["Physical health", "Mental health", "Daily functioning", "Unspecific", "None"])),
      ),
      rg.LabelQuestion(
          name="frequency",
          title="The concept 'frequency' is mentioned.",
          labels=dict(zip(codes_freq, ["Yes", "No"])),
      ),
    ],
    metadata=[
        # later we only want to train on the latest batches
        rg.IntegerMetadataProperty(
            name="batch_id",
            min=0,
            max=1000,
            title="batch_id"
        ),
        rg.IntegerMetadataProperty(
            name="small-text-id",
            min=0,
            max=100000,
            title="Small Text ID"
        ),
    ],
)

n_sets = 3  # number of subsets you want

datasets = []
for i in range(1, n_sets + 1):
    ds = rg.Dataset(
        name=f"train_subset_{i}",
        settings=settings,
    )
    ds.create()
    datasets.append(ds)

/home/sc.uni-leipzig.de/pk35momo/.local/lib/python3.12/site-packages/argilla/datasets/_resource.py:264: UserWarning: Workspace not provided. Using default workspace: default id: c0cca5f2-fd97-4375-8f7c-61e05c36dccc
  warnings.warn(f"Workspace not provided. Using default workspace: {workspace.name} id: {workspace.id}")
/home/sc.uni-leipzig.de/pk35momo/.local/lib/python3.12/site-packages/argilla/_resource.py:136: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self._last_api_call = datetime.utcnow()


In [8]:
import argilla as rg

current_batch_id = 0

records = [[
    rg.Record(
        fields={"childPart":row.get('childPart', ""),
                "interview": {"text": row.get('interview', "")}
               },
        metadata={"batch_id":current_batch_id,
                  "small-text-id": i}
    )  
    for i, row in df.iterrows() 
] for df in parts]

for ds, recs in zip(datasets, records):
    ds.records.log(recs)

DatasetRecords: The provided batch size 256 was normalized. Using value 70.

Sending records...: 100%|██████████| 1/1 [00:00<00:00,  1.18batch/s]


DatasetRecords: The provided batch size 256 was normalized. Using value 70.

Sending records...: 100%|██████████| 1/1 [00:00<00:00,  1.31batch/s]


DatasetRecords: The provided batch size 256 was normalized. Using value 70.

Sending records...: 100%|██████████| 1/1 [00:00<00:00,  1.26batch/s]


In [10]:
client.users("aisic")

User(id=UUID('2098bfdb-3d65-4288-90b9-a1b84ad6cf17') inserted_at=datetime.datetime(2025, 7, 10, 12, 38, 28, 234177) updated_at=datetime.datetime(2025, 7, 10, 12, 38, 28, 234177) username='aisic' role=<Role.owner: 'owner'> first_name='aisic' last_name=None password=None)

In [11]:
rename_dict = {
    '22cb6f61-3095-4765-a459-55288f72aeaa': 'peter',
    '55c6dea3-9a8f-4cbb-b2a8-6af210558098': 'leo',
    'ac1d45f0-a2b9-482e-a4eb-705753d754ef': 'nico',
    '2098bfdb-3d65-4288-90b9-a1b84ad6cf17': 'peter'
}

In [13]:
status_filter = rg.Query(
    filter=rg.Filter([("response.status", "==", "submitted")])
)

def get_ordered_submitted(ds, status_filter):
    # Step 1: get submitted records
    submitted = ds.records(status_filter).to_list(flatten=True)

    # Step 2: lookup table by small-text-id
    lookup = {rec["small-text-id"]: rec for rec in submitted}

    # Step 3: reorder to logged order
    ordered = [lookup[uid] for uid in range(len(train_df)) if uid in lookup]

    return ordered

# Apply to every dataset automatically
ordered_submitted_lists = [
    get_ordered_submitted(ds, status_filter)
    for ds in datasets
]

# Unpack if you want the three lists separately:
# ordered_list1, ordered_list2, ordered_list3 = ordered_submitted_lists

In [14]:
def get_labels_with_annotators(records, code_neg, code_full):

    results = []

    for record in records:

        annotators = record["health.responses.users"]
        health_lists = record["health.responses"]
        freq_labels = record["frequency.responses"]
        idx = record["small-text-id"]

        record_output = []

        for user, h_list, f in zip(annotators, health_lists, freq_labels):

            # merge
            labels = list(h_list) + [f]

            # remove negatives
            labels = [lbl for lbl in labels if lbl not in code_neg]

            # name → full index
            full_idx = [code_full[lbl] for lbl in labels]

            # full → positive continuous
            pos_idx = [full_to_pos[i] for i in full_idx if i in full_to_pos]

            record_output.append({
                "small-text-id": idx,
                "annotator": user,
                "labels": pos_idx
            })

        results.append(record_output)

    return results
    
y_expert_train = [get_labels_with_annotators(ls, code_neg, code_full) for ls in ordered_submitted_lists]

In [22]:
flat[1]

[{'small-text-id': 83,
  'annotator': '22cb6f61-3095-4765-a459-55288f72aeaa',
  'labels': [0, 1, 2, 4]}]

In [24]:
flat = [
    item
    for annotator_block in y_expert_train
    for single_list in annotator_block
    for item in single_list
]

In [38]:
y_expert_test[1]

[{'small-text-id': 83,
  'annotator': '22cb6f61-3095-4765-a459-55288f72aeaa',
  'labels': [0, 1, 2, 4]}]

In [25]:
for i in range(len(flat)):
    l = flat[i]
    train_df.at[l["small-text-id"], "annotator"] = rename_dict[l["annotator"]]
    train_df.at[l["small-text-id"], "label"] = l["labels"]

In [28]:
sum(train_df['annotator'] == 'leo')

70

In [29]:
sum(train_df['annotator'] == 'peter')

139

In [30]:
train_df[train_df['annotator'] != '']

,id,model,interviewer_temperature,child_temperature,child_sex,child_name,child_health_status,interviewer_prompt,child_prompt,num_questions,stop_on_phrase,interview_id,speech_index,childPart,interview,label,labeled,batch_id,experiment,annotator
1,536,meta-llama-3.1-8b-instruct,1.0,1.0,Mädchen,Soey,schlecht,"Du bist eine freundliche und geduldige Person,...","Du bist ein Kind, das zu seinem Gesundheitszus...",20,True,interview_20251011_082407.json,4,Mein Gesundheitszustand ist schlecht... Ich ha...,"<span style=""color: lightgrey;"">Interviewerin:...","[0, 2, 4]",False,99,,leo
23,514,meta-llama-3.1-8b-instruct,1.0,1.0,Junge,Finn,gut,"Du bist eine freundliche und geduldige Person,...","Du bist ein Kind, das zu seinem Gesundheitszus...",20,True,interview_20251011_074911.json,0,Ich heiße Finn! Ich bin sechs Jahre alt.,"<span style=""color: lightgrey;""></span>\n ...",[],False,99,,peter
75,180,openai-gpt-oss-120b,0.5,1.0,Mädchen,Merle,sehr schlecht,"Du bist eine freundliche und geduldige Person,...","Du bist ein Kind, das zu seinem Gesundheitszus...",20,True,interview_20251010_215410.json,8,Also weil ich fast immer Schmerzen habe. Mein ...,"<span style=""color: lightgrey;"">Interviewerin:...","[0, 1, 4]",False,99,,peter
83,41,openai-gpt-oss-120b,0.0,0.5,Junge,Leon,schlecht,"Du bist eine freundliche und geduldige Person,...","Du bist ein Kind, das zu seinem Gesundheitszus...",20,True,interview_20251010_175324.json,16,"„Gesundheit“ bedeutet für mich, dass ich mich ...","<span style=""color: lightgrey;"">Interviewerin:...","[0, 1, 2, 4]",False,99,,peter
116,487,meta-llama-3.1-8b-instruct,1.0,0.5,Junge,Finn,mittelmäßig,"Du bist eine freundliche und geduldige Person,...","Du bist ein Kind, das zu seinem Gesundheitszus...",20,True,interview_20251011_070405.json,6,"Mein Gesundheitszustand... Ich denke, ich bin ...","<span style=""color: lightgrey;"">Interviewerin:...","[0, 4]",False,99,,peter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5510,478,meta-llama-3.1-8b-instruct,1.0,0.0,Mädchen,Leonie,sehr schlecht,"Du bist eine freundliche und geduldige Person,...","Du bist ein Kind, das zu seinem Gesundheitszus...",20,True,interview_20251011_065007.json,16,"""Gesundheit"" bedeutet für mich, dass ich mich ...","<span style=""color: lightgrey;"">Interviewerin:...","[1, 2, 3, 4]",False,99,,peter
5511,656,llama-3.1-sauerkrautlm-70b-instruct,0.5,0.0,Mädchen,Soey,schlecht,"Du bist eine freundliche und geduldige Person,...","Du bist ein Kind, das zu seinem Gesundheitszus...",20,True,interview_20251013_125630.json,12,"Ich denke, wenn ich nicht so oft krank wäre un...","<span style=""color: lightgrey;"">Interviewerin:...","[2, 3, 4]",False,99,,peter
5513,649,llama-3.1-sauerkrautlm-70b-instruct,0.5,0.0,Mädchen,Leonie,gut,"Du bist eine freundliche und geduldige Person,...","Du bist ein Kind, das zu seinem Gesundheitszus...",20,True,interview_20251013_124518.json,6,"Ich dachte daran, dass ich selten zum Arzt mus...","<span style=""color: lightgrey;"">Interviewerin:...","[2, 4]",False,99,,peter
5550,532,meta-llama-3.1-8b-instruct,1.0,1.0,Mädchen,Leonie,mittelmäßig,"Du bist eine freundliche und geduldige Person,...","Du bist ein Kind, das zu seinem Gesundheitszus...",20,True,interview_20251011_081705.json,4,"Ich würde sagen, mittelmäßig. Ich bin nicht im...","<span style=""color: lightgrey;"">Interviewerin:...","[0, 4]",False,99,,peter


In [31]:
train_df.to_pickle("data/train_df_new_interviews.pkl")

In [32]:
import pandas as pd 
train_df = pd.read_pickle("data/train_df_new_interviews.pkl")
train_df.to_csv("data/train_df_new_interviews.csv")